# Loop 10 Analysis: Final Decision

## Key Questions:
1. Should we submit exp_010 ([16]) or keep exp_007 ([32,16])?
2. What is the CV-LB relationship telling us?
3. Is the target (0.0333) achievable?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.011081, 'lb': 0.09816, 'arch': '[128,128,64]'},
    {'exp': 'exp_001', 'cv': 0.012297, 'lb': 0.10649, 'arch': 'LightGBM'},
    {'exp': 'exp_003', 'cv': 0.010501, 'lb': 0.09719, 'arch': '[256,128,64]'},
    {'exp': 'exp_005', 'cv': 0.010430, 'lb': 0.09691, 'arch': '[256,128,64] 15-bag'},
    {'exp': 'exp_006', 'cv': 0.009749, 'lb': 0.09457, 'arch': '[64,32]'},
    {'exp': 'exp_007', 'cv': 0.009262, 'lb': 0.09316, 'arch': '[32,16]'},
]

df = pd.DataFrame(submissions)
df['ratio'] = df['lb'] / df['cv']
df['lb_improvement'] = df['lb'].diff() * -1  # Negative diff = improvement
df['cv_improvement'] = df['cv'].diff() * -1
print(df.to_string())

In [ ]:
# Analyze the CV-LB relationship
print('=== CV-LB Relationship Analysis ===')
print(f'\nCorrelation between CV and LB: {df["cv"].corr(df["lb"]):.4f}')

# Linear regression to predict LB from CV
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f'\nLinear fit: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')

# Predict LB for exp_010 (CV = 0.009192)
cv_010 = 0.009192
predicted_lb_010 = slope * cv_010 + intercept
print(f'\nPredicted LB for exp_010 (CV={cv_010}): {predicted_lb_010:.4f}')

# Using ratio method
avg_ratio = df['ratio'].mean()
ratio_predicted_lb = cv_010 * avg_ratio
print(f'Using average ratio ({avg_ratio:.2f}x): {ratio_predicted_lb:.4f}')

# Using latest ratio (most pessimistic)
latest_ratio = df['ratio'].iloc[-1]
latest_predicted_lb = cv_010 * latest_ratio
print(f'Using latest ratio ({latest_ratio:.2f}x): {latest_predicted_lb:.4f}')

In [ ]:
# Plot CV vs LB
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], s=100, c='blue', alpha=0.7)

# Add labels
for i, row in df.iterrows():
    plt.annotate(row['arch'], (row['cv'], row['lb']), textcoords='offset points', xytext=(5,5))

# Add regression line
x_line = np.linspace(0.008, 0.013, 100)
y_line = slope * x_line + intercept
plt.plot(x_line, y_line, 'r--', label=f'Linear fit (R²={r_value**2:.3f})')

# Add exp_010 prediction
plt.scatter([cv_010], [predicted_lb_010], s=150, c='green', marker='*', label=f'exp_010 predicted: {predicted_lb_010:.4f}')

# Add target line
plt.axhline(y=0.0333, color='red', linestyle=':', label='Target: 0.0333')

plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV vs LB Relationship')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('/home/code/exploration/cv_lb_analysis.png', dpi=100, bbox_inches='tight')
plt.show()
print('Plot saved to /home/code/exploration/cv_lb_analysis.png')

In [ ]:
# Decision analysis
print('=== DECISION ANALYSIS ===')
print(f'\nCurrent best LB: {df["lb"].min():.4f} (exp_007, [32,16])')
print(f'exp_010 CV: {cv_010}')
print(f'exp_010 predicted LB: {predicted_lb_010:.4f} (linear fit)')
print(f'exp_010 predicted LB: {latest_predicted_lb:.4f} (latest ratio)')

# Calculate expected improvement
exp_improvement_linear = df['lb'].min() - predicted_lb_010
exp_improvement_ratio = df['lb'].min() - latest_predicted_lb
print(f'\nExpected LB improvement (linear): {exp_improvement_linear:.4f} ({exp_improvement_linear/df["lb"].min()*100:.1f}%)')
print(f'Expected LB improvement (ratio): {exp_improvement_ratio:.4f} ({exp_improvement_ratio/df["lb"].min()*100:.1f}%)')

# Risk assessment
print('\n=== RISK ASSESSMENT ===')
print('1. CV-LB ratio is INCREASING (8.86x -> 10.06x)')
print('   This suggests diminishing returns on LB as CV improves')
print('2. The linear fit predicts modest improvement')
print('3. The ratio method predicts minimal/no improvement')
print('\n=== RECOMMENDATION ===')
if predicted_lb_010 < df['lb'].min():
    print(f'SUBMIT exp_010: Linear fit predicts improvement ({predicted_lb_010:.4f} < {df["lb"].min():.4f})')
else:
    print(f'KEEP exp_007: No significant improvement expected')

In [ ]:
# Final summary
print('\n' + '='*60)
print('FINAL SUMMARY')
print('='*60)
print(f'\nTarget: 0.0333')
print(f'Best LB achieved: {df["lb"].min():.4f} (exp_007)')
print(f'Gap to target: {(df["lb"].min() - 0.0333)/0.0333*100:.1f}% worse')
print(f'\nexp_010 ([16]) CV: {cv_010}')
print(f'exp_010 predicted LB range: {latest_predicted_lb:.4f} - {predicted_lb_010:.4f}')
print(f'\nKey insight: The target (0.0333) is UNREACHABLE with tabular MLP.')
print('To reach target, would need CV ≈ 0.0036 (61% improvement from current best)')
print('\nThe competition is about maximizing relative ranking, not beating target.')
print('exp_010 has the best CV and should be submitted for the final slot.')

In [ ]:
# Verify submission file format
print('\n=== SUBMISSION FILE VERIFICATION ===')
sub = pd.read_csv('/home/submission/submission.csv')
print(f'Shape: {sub.shape}')
print(f'Columns: {list(sub.columns)}')
print(f'\nTask distribution:')
print(sub['task'].value_counts())
print(f'\nFold distribution for task 0 (single solvent):')
print(sub[sub['task']==0]['fold'].value_counts().sort_index())
print(f'\nFold distribution for task 1 (full data):')
print(sub[sub['task']==1]['fold'].value_counts().sort_index())